In [ ]:
##########################################################################################
#####             Split Plot to smaller chunk with lenght of max token = 50
##########################################################################################
max_tokens = 50
# Function to split the text into chunks of a maximum number of tokens
def split_into_many(text, max_tokens = max_tokens):
    # Split the text into sentences
    #print('\n**************', 'in split to many ', '******************\n')
    sentences = text.split('. ')
    # Get the number of tokens for each sentence
    #n_tokens = [len(tokenizer(" " + sentence, padding=True, truncation=False, return_tensors='pt')[0]) for sentence in sentences]
    n_tokens = [len([token.text for token in nlp(sentence)]) for sentence in sentences]
    chunks = []
    tokens_so_far = 0
    chunk = []
    # Loop through the sentences and tokens joined together in a tuple
    for sentence, token in zip(sentences, n_tokens):
        # If the number of tokens so far plus the number of tokens in the current sentence is greater
        # than the max number of tokens, then add the chunk to the list of chunks and reset the chunk and tokens so far
        if tokens_so_far + token > max_tokens:
            chunks.append(". ".join(chunk) + ".")
            chunk = []
            tokens_so_far = 0
        # If the number of tokens in the current sentence is greater than the max number of
        # tokens, go to the next sentence
        if token > max_tokens:
            continue
        # Otherwise, add the sentence to the chunk and add the number of tokens to the total
        chunk.append(sentence)
        tokens_so_far += token + 1
    # Add the last chunk to the list of chunks
    if chunk:
        chunks.append(". ".join(chunk) + ".")
    return chunks

In [ ]:
##########################################################################################
#####                             Clean text to the word tokens
##########################################################################################
import spacy
nlp = spacy.load('en_core_web_sm')
def cleanup_text(docs):
    texts = []
    #counter = 1
    for doc in docs:
      #counter += 1
      doc = nlp(doc)# nlp(doc,disable=['parser', 'ner'])
      tokens = [tok.lemma_.lower().strip() for tok in doc if tok.lemma_ != '-PRON-' and not tok.is_stop and not tok.like_num and  not tok.is_punct]
      tokens = ' '.join(tokens)
      texts.append(tokens)
    return texts

In [ ]:
######################################################################################################
#####        create proper Structure for each Query containing quey, Top_k, chunks of plots
######################################################################################################
query_info = []
for q in range(len(queries)):
  query_json = {}
  chunks_top_k = []
  query_json.update({"query": queries[q],"top_k": {"score_idx": all_q_results[q], "chunks": []}})
  chunks = []
  for r in all_q_results[q][1]:
    # If the text is None, go to the next row
    if df_new.iloc[int(r)]['Plot'] is None:
        continue
    chunks = split_into_many(df_new.iloc[int(r)]['Plot'])
    clean_text = cleanup_text(chunks)
    chunks_top_k.append(clean_text)
  query_json["top_k"]["chunks"]=chunks_top_k
  query_info.append(query_json)